In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
cd /content/drive/MyDrive/Twente/Q2/RPCN---November-2025

/content/drive/MyDrive/Twente/Q2/RPCN---November-2025


In [16]:
#!/usr/bin/env python3
import os
import pandas as pd
import numpy as np

def calibrate_imu_series(directory: str, g: float = 9.80665):
    """
    Calibrate an IMU from 18 CSV files (space-separated, headers as given).

    Files must be named like  MT_001-000.csv , MT_002-000.csv , ... MT_018-000.csv
    (the numeric part is used for ordering).

    Returns a dict with per-series and overall results.
    """
    # ------------------------------------------------------------
    # 1. List & sort files
    # ------------------------------------------------------------
    csv_files = [f for f in os.listdir(directory) if f.lower().endswith('.csv')]
    if len(csv_files) != 18:
        raise ValueError(f"Expected 18 CSV files, found {len(csv_files)} in {directory}")

    # sort by the number in the filename (001 … 018)
    csv_files.sort(key=lambda x: int(x.split('_')[-1].split('-')[0]))

    # ------------------------------------------------------------
    # 2. Helper: average of a column in one file
    # ------------------------------------------------------------
    def avg_column(filepath: str, col: str) -> float:
        df = pd.read_csv(filepath, sep=r'\s+', usecols=[col])
        return float(df[col].mean())

    # ------------------------------------------------------------
    # 3. Define which files belong to which orientation
    # ------------------------------------------------------------
    # 0-based indices for the three series (0-5, 6-11, 12-17)
    series_idxs = [(0, 6, 12), (1, 7, 13), (2, 8, 14), (3, 9, 15), (4, 10, 16), (5, 11, 17)]

    # Axis → (up indices, down indices, column name)
    axis_cfg = {
        'Z': ( (1, 7, 13), (0, 6, 12), 'Acc_Z' ),   # 001-down, 002-up
        'X': ( (2, 8, 14), (3, 9, 15), 'Acc_Y' ),   # 003-down, 004-up
        'Y': ( (4, 10, 16), (5, 11, 17), 'Acc_X' ) # 006-down, 005-up
    }

    # ------------------------------------------------------------
    # 4. Compute per-series values
    # ------------------------------------------------------------
    results = {'Z': [], 'X': [], 'Y': []}

    for axis, (up_idx, down_idx, col) in axis_cfg.items():
        for ser in range(3):                     # three series
            # files for this series, this orientation
            up_files   = [os.path.join(directory, csv_files[i]) for i in up_idx[ser:ser+1]]
            down_files = [os.path.join(directory, csv_files[i]) for i in down_idx[ser:ser+1]]

            f_up   = avg_column(up_files[0], col)
            f_down = avg_column(down_files[0], col)

            b = (f_up + f_down) / 2.0
            S = (f_up - f_down - 2 * g) / (2 * g)

            results[axis].append({'series': ser + 1, 'f_up': f_up, 'f_down': f_down,
                                  'b': b, 'S': S})

    # ------------------------------------------------------------
    # 5. Overall average
    # ------------------------------------------------------------
    overall = {}
    for axis in results:
        b_vals = [r['b'] for r in results[axis]]
        S_vals = [r['S'] for r in results[axis]]
        overall[axis] = {'b_avg': np.mean(b_vals), 'S_avg': np.mean(S_vals)}

    # ------------------------------------------------------------
    # 6. Pretty printing
    # ------------------------------------------------------------
    def print_series(axis: str):
        print(f"\n=== {axis}-axis calibration ===")
        print(f"{'Series':>6}  {'f_up':>10}  {'f_down':>10}  {'b':>12}  {'S':>12}")
        print("-" * 52)
        for r in results[axis]:
            print(f"{r['series']:6d}  {r['f_up']:10.6f}  {r['f_down']:10.6f}  "
                  f"{r['b']:12.6f}  {r['S']:12.6f}")
        print("-" * 52)
        print(f"{'AVG':>6}  {'':>10}  {'':>10}  {overall[axis]['b_avg']:12.6f}  {overall[axis]['S_avg']:12.6f}")

    print_series('Z')
    print_series('X')
    print_series('Y')

    # ------------------------------------------------------------
    # 7. Return structured data (optional)
    # ------------------------------------------------------------
    return {'per_series': results, 'overall': overall}


calibrate_imu_series('/content/drive/MyDrive/Twente/Q2/RPCN---November-2025/Assignment 1/Data/csv')



=== Z-axis calibration ===
Series        f_up      f_down             b             S
----------------------------------------------------
     1   -9.818758    9.806066     -0.006346     -2.000588
     2   -9.816788    9.806664     -0.005062     -2.000518
     3   -9.822028    9.803234     -0.009397     -2.000610
----------------------------------------------------
   AVG                             -0.006935     -2.000572

=== X-axis calibration ===
Series        f_up      f_down             b             S
----------------------------------------------------
     1   -9.745906    9.868271      0.061183     -2.000045
     2   -9.747658    9.869006      0.060674     -2.000172
     3   -9.747367    9.867701      0.060167     -2.000090
----------------------------------------------------
   AVG                              0.060675     -2.000102

=== Y-axis calibration ===
Series        f_up      f_down             b             S
----------------------------------------------------
  

{'per_series': {'Z': [{'series': 1,
    'f_up': -9.818758250835423,
    'f_down': 9.80606623488416,
    'b': -0.006346007975631451,
    'S': -2.0005875852467248},
   {'series': 2,
    'f_up': -9.816787849124381,
    'f_down': 9.806663514657263,
    'b': -0.005062167233559123,
    'S': -2.000517575511599},
   {'series': 3,
    'f_up': -9.822028428029546,
    'f_down': 9.80323360773231,
    'b': -0.009397410148618057,
    'S': -2.0006098940903296}],
  'X': [{'series': 1,
    'f_up': -9.745906166734777,
    'f_down': 9.868271201150565,
    'b': 0.061182517207893916,
    'S': -2.0000447333128717},
   {'series': 2,
    'f_up': -9.747658021970555,
    'f_down': 9.86900603359056,
    'b': 0.06067400581000282,
    'S': -2.0001715190998515},
   {'series': 3,
    'f_up': -9.74736670509735,
    'f_down': 9.867701250223615,
    'b': 0.060167272563131924,
    'S': -2.0000901406352303}],
  'Y': [{'series': 1,
    'f_up': -9.850110873937957,
    'f_down': 9.779993890153083,
    'b': -0.03505849189243